<a href="https://colab.research.google.com/github/tguerand/SentimentAnalysis/blob/master/bin/data_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import os
import urllib.request
print("Tensorflow version is :", tf.__version__)

Tensorflow version is : 2.3.0


#Access Path

In [2]:
!git clone https://github.com/tguerand/SentimentAnalysis
os.mkdir("./SentimentAnalysis/DATA")
#Download file
!gdown --id 1dO4aubOro159awBPzD675c22qtXMOVp0 --output ./SentimentAnalysis/DATA/training.csv

Cloning into 'SentimentAnalysis'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 23 (delta 3), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), done.
Downloading...
From: https://drive.google.com/uc?id=1dO4aubOro159awBPzD675c22qtXMOVp0
To: /content/SentimentAnalysis/DATA/training.csv
239MB [00:03, 77.7MB/s]


#Import Data

In [9]:
df=pd.read_csv("./SentimentAnalysis/DATA/training.csv",encoding='latin-1')
df.columns=['target', 't_id', 'created_at', 'query', 'user', 'text']
#df.head()

In [10]:
print(df["text"].head(5).to_string())

0    is upset that he can't update his Facebook by ...
1    @Kenichan I dived many times for the ball. Man...
2      my whole body feels itchy and like its on fire 
3    @nationwideclass no, it's not behaving at all....
4                        @Kwesidei not the whole crew 


In [ ]:
sentences = df['text']

In [ ]:
tokenizer = Tokenizer(num_words=1000)

In [ ]:
tokenizer.fit_on_texts(sentences)
tokenizer.word_index

{'i': 1,
 'to': 2,
 'the': 3,
 'a': 4,
 'my': 5,
 'and': 6,
 'you': 7,
 'is': 8,
 'it': 9,
 'in': 10,
 'for': 11,
 'of': 12,
 'on': 13,
 'me': 14,
 'so': 15,
 'have': 16,
 'that': 17,
 'but': 18,
 "i'm": 19,
 'just': 20,
 'with': 21,
 'be': 22,
 'at': 23,
 'not': 24,
 'was': 25,
 'this': 26,
 'now': 27,
 'good': 28,
 'up': 29,
 'day': 30,
 'out': 31,
 'all': 32,
 'get': 33,
 'like': 34,
 'are': 35,
 'no': 36,
 'go': 37,
 'quot': 38,
 'http': 39,
 'today': 40,
 'do': 41,
 "it's": 42,
 'too': 43,
 'your': 44,
 'work': 45,
 'love': 46,
 'going': 47,
 'got': 48,
 'lol': 49,
 'time': 50,
 'back': 51,
 'from': 52,
 'u': 53,
 'one': 54,
 'what': 55,
 'com': 56,
 'will': 57,
 'im': 58,
 'know': 59,
 'we': 60,
 'about': 61,
 'am': 62,
 'really': 63,
 "don't": 64,
 'amp': 65,
 'had': 66,
 'can': 67,
 'see': 68,
 "can't": 69,
 'some': 70,
 'its': 71,
 'if': 72,
 'still': 73,
 '2': 74,
 'well': 75,
 'night': 76,
 'new': 77,
 'want': 78,
 'how': 79,
 'think': 80,
 'home': 81,
 'thanks': 82,
 'oh': 